<a href="https://colab.research.google.com/github/tinySculpture/STINTSY-Project/blob/master/Project_Labor_Force_Survey_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> ## **Group 8 - STINTSY S17**
> Members:
> - CRUZ, GIOVANNI JONATHAN
> - LIM, NATHAN MIGUEL
> - LUISTRO, JOSIAH MARI
> - MANGAWANG, FELIX MELFORD

---

# Introduction

## Labor Force Survey 2016

Taken from the Philippine Statistics Authority (PSA), the dataset is a survey conducted in 2016 with respect in Demographic, Economic Characteristics and Past Quarter Activities of the surveyed individuals in this dataset.

The group picked this dataset over the family dataset because of its recency (2016 over 2012), simplicity of raw data (50 columns instead of 116), and most importantly a general group interest on the labor force dataset as a whole.

## Task

With this, the group decided to make a classification task with a target variable of `PUFC23_PCLASS`. This variable is described on the website as the Class of Worker on their Primary Occupation.

This variable has 6 different values listed in the value set, namely:
0. Worked for private household
1. Worked for private establishment
2. Worked for government/government corporation
3. Self-employed without any paid employee
4. Employer in own family-operated farm or business
5. Worked with pay on own family-operated farm or business
6. Worked without pay on own family-operated farm or business

To briefly explain, the variable is the catgory of the person's relationship to the establishment he works in. According to the [data dictionary](https://psada.psa.gov.ph/catalog/67/data-dictionary/F1?file_name=lfs_april2016), this is referred to as the "Industrial Status" in other countries.

# Description of the Dataset

The survey covers individuals aged 15 years and older and provides insights into labor market trends at both national and regional levels (17 administrative regions). Key variables include employment status, industry, occupation, hours worked, and demographic details like age, sex, and education.

The data was collected through face-to-face interviews administered to 42,768 sample households. Considering how many people are in a single household, the total of 180,862 data points makes sense.

The 2013 Master Sample (MS) was used, with Primary Sampling Units (PSUs) stratified by geographic location, household wealth, and overseas worker prevalence.

The dataset has different key characteristics that can be inferred:
> Demographic Characteristics:
>
> - Household ID
> - Age
> - Sex
> - Marital Status
> - Highest Grade Completed
> - Overseas Worker Indicator

> Economic Characteristics:
>
> - Employment Status
> - Industry Group
> - Occupation
> - Hours Worked
> - Job Search Method


> Geographic Coverage:
>
> - Region
> - Urban/Rural

> Other Notable Features:
>
> - Class of Worker
> - Household Head Indicators



# Python libraries and modules used.


| LIBRARY | PURPOSE |
| ------- | ------- |
| pandas | Data table manipulation |
| numpy | Matrix operations |
| ydata_profiling | Create a report for easier EDA |
| sklearn | Mostly used for preprocessing, also used for making models |
| matplotlib | Plot graphs and visualize data |
| imblearn | To do SMOTE on imbalanced data |

> ⚠ **Run these when using Google Colab**

In [ ]:
# Run for google colab
!git clone https://github.com/tinySculpture/STINTSY-Project.git
%cd STINTSY-Project/

In [ ]:
%cd STINTSY-Project/

In [ ]:
!pip install ydata-profiling --quiet

In [ ]:
!pip install --quiet optuna

In [ ]:
import optuna
print(optuna.__version__)

> ⚠ End of Cells for Google Colab

In [ ]:
%load_ext autoreload
%autoreload 2

# Import libraries
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE

from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import ttest_ind

from random_forest_model import preprocess_data, train_random_forest, evaluate_model
from statistical_test import get_statistical_significance

# Data Preprocessing

## Read dataset

In [ ]:
df = pd.read_csv("LFS PUF April 2016.CSV")

## Create a Profile Report
Using ydata-profiling, we can generate a profile report to get an overview of the dataset. A function can be defined to generate the report and save it as an HTML file.

In [ ]:
def generate_report(dataframe, filename=""):
    report = ProfileReport(dataframe)
    report.to_file(filename + ".html")

In [ ]:
generate_report(df, 'report-raw')

### Data Cleaning

1. Force Missing Values to NaN to correctly identify missing values because of different data types. (Int, Float, and Object)
2. Delete duplicated rows (keep the first instance)

There are missing data that are represented as different invisible characters - replace them with an empty string

In [ ]:
df.isna().sum()

In [ ]:
# df.replace(["", " ", "NA", "N/A", "-", "NULL"], np.nan, inplace=True)
df.replace("", np.nan, inplace=True)

In [ ]:
df = df.map(lambda x: np.nan if isinstance(x, str) and x.strip() == "" else x)

### Sanity Checking
Check if missing data is properly replaced with NaN

In [ ]:
df

### Impute missing values

- Fill numerical columns with their *mean*.
- Fill categorical values with their *mode*.

In [ ]:
# Replace numeric values with mean
df.fillna(df.select_dtypes(include=["number"]).mean(numeric_only=True), inplace=True)

# Replace categorical values with mode
for col in df.select_dtypes(include=["object"]):
    mode_value = df[col].mode()[0] if not df[col].mode().empty else "unknown"
    df[col] = df[col].fillna(mode_value)

df

### Delete Unnecessary Columns

- Delete constant values (Month, Year)
    - PUFSVYYR
    - PUFSVYMO
- Delete IDs
    - PUFPRRCD (-0.035)
    - PUFHHNUM (0.021)
    - PUFREG (0.020)
- Delete Columns with low correlation to target variable (Within -0.05 to +0.05)
    - PUF10_CONWR (0.037)
    - PUFC14_PROCC (-0.490)
    - PUFC25_PBASIC (0.035)
    - PUFC27_NJOBS (0.035)
    - PUFC31_FLWRK (0.026)
    - PUFC32_JOBSM (0.031)
    - PUFC33_WEEKS (-0.014)
    - PUFC34_WYNOT (-0.019)
    - PUFC35_LTLOOKW (0.026)
    - PUFC37_WILLING (0.046)
    - PUFPRV (-0.035)
    - PUFRPL (0.004)
- Delete data not useful for prediction
    - PUFC41_WQTR - Work in the previous quarter
    - PUFC43_QKB - Kind of business in the previous quarter
    - PUFC40_POCC - Previous occupation if unemployed (we need employment class)
    - PUFWGTFIN - Weight used for surveying
    - PUFPSU - Primary Sampling Unit (PSU) useful for survey design but not for prediction
    - PUFC12_JOB - PUFC11_WORK is more useful
    - PUFC36_AVAIL - Availability for work (useful for unemployed people)
    - PUFC19_PHOURS - Hours worked in the previous week
    - PUFC38_PREVJOB - Previous job if employed (we need current job)

In [ ]:
drop_cols = [
    # IDs, Constants
    "PUFSVYYR", "PUFSVYMO", "PUFPRRCD", "PUFHHNUM", "PUFREG",
    # Low correlation
    "PUFC01_LNO", "PUFC10_CONWR", "PUFC14_PROCC", "PUFC25_PBASIC", "PUFC27_NJOBS", "PUFC31_FLWRK", "PUFC32_JOBSM", "PUFC33_WEEKS", "PUFC34_WYNOT", "PUFC35_LTLOOKW", "PUFC37_WILLING", "PUFPRV", "PUFRPL",
    # Not useful for prediction of type of worker
    "PUFC41_WQTR", "PUFC43_QKB", "PUFC40_POCC", "PUFPWGTFIN", "PUFPSU", "PUFC12_JOB", "PUFC36_AVAIL", "PUFC19_PHOURS", "PUFC38_PREVJOB"
]
df.drop(columns=drop_cols, inplace=True)

In [ ]:
df

### Recheck EDA

Check if the data has been changed significantly, and if the statistical tests are still the same after imputation and feature reduction.

In [ ]:
generate_report(df, "report-cleaned")

After checking the EDA again, duplicate data is present since there are columns that have been deleted.

In [ ]:
print("Duplicates: ", df.duplicated().sum())

In [ ]:
df = df.drop_duplicates()
print("Duplicates after dropping: ", df.duplicated().sum())

#### Binning and Encoding

- One-hot encode categorical values

In [ ]:
# One hot encode binary categorical columns
cat_cols = ['PUFURB2K10', 'PUFC04_SEX', 'PUFC08_CURSCH', 'PUFC09_GRADTECH', 'PUFC11_WORK', 'PUFC20_PWMORE', "PUFC21_PLADDW", 'PUFC22_PFWRK', 'PUFC26_OJOB', "PUFC30_LOOKW"]

df = pd.get_dummies(df, columns=cat_cols, drop_first=True, dtype=float)
df = df.rename(
    columns = {
        "PUFURB2K10_2": "Is_Rural",
        "PUFC04_SEX_2": "Is_Female",
        "PUFC08_CURSCH_2": "Is_Not_In_School",
        "PUFC09_GRADTECH_2": "Is_Not_In_Techvoc",
        "PUFC11_WORK_2": "Is_Not_Working",
        "PUFC20_PWMORE_2": "Has_Not_Wanted_More_Hours",
        "PUFC21_PLADDW_2": "Has_Not_Looked_For_Work",
        "PUFC22_PFWRK_2": "Is_Not_First_Time",
        "PUFC26_OJOB_2": "No_Other_Job",
        "PUFC30_LOOKW_2": "Has_Not_Looked_For_Work_Week"
    }
)

In [ ]:
def one_hot_encode(df, df_col, df_new_col, category_map):
    df[df_new_col] = df[df_col].apply(category_map)

    # One hot encode the new categories
    encoder = OneHotEncoder(sparse_output=False, drop="first")
    encoded = encoder.fit_transform(df[[df_new_col]])
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([df_new_col]), index=df.index)

    return pd.concat([df, encoded_df], axis=1)

In [ ]:
df

#### PUFC07_GRADE
This classification data contains a lot of values in the value set. It can be noticed that there are different values for highest educational attainment and the program category. Since they are only one column, it might be better to encode the highest educational attainment as an ordinal encoded variable, and the program category as 15 different categories instead of 50+.

In [ ]:
education_categories = {
    "Primary": ["000", "010", "210", "220", "230", "240", "250", "260", "280"],
    "Secondary": ["310", "320", "330", "340", "350"],
    "Post-Secondary": ["410", "420", "810", "820", "830", "840", "900"],
    "STEM": ["542", "544", "548", "552", "554", "558", "562", "564", "572", "642", "644", "646", "648", "652", "654", "658", "662", "664", "672"],
    "Business": ["534", "634", "638"],
    "Art": ["521", "522", "531", "532", "621", "622", "631", "632"],
    "Other": ["501", "508", "509", "514", "576", "581", "584", "585", "586", "601", "614", "676", "681", "684", "685", "686", "589", "689"]
}


def get_education_category(x):
    for key, values in education_categories.items():
        if x in values:
            return key
    return "Unknown"

In [ ]:
df = one_hot_encode(df, "PUFC07_GRADE", "Education", get_education_category)

### PUFC16_PKB
The valueset offers a binning for this column already. The values are binned into 3 categories: Agriculture, Industry, Services.

In [ ]:
industry_categories = {
    "Agriculture": range(1, 3),
    "Industry": range(5, 43),
    "Services": range(45, 99)
}

def get_industry_category(x):
    for key, values in industry_categories.items():
        if x in values:
            return key
    return "Unknown"

In [ ]:
df = one_hot_encode(df, "PUFC16_PKB", "KindOfBusiness", get_industry_category)

### PUFC29_WWM48H
One hot encode based on the given reasons for working more than 48 hours.

1. Want - Wanted more earnings
2. Requirement - Requirements of the job
3. Exceptional - Exceptional week
4. Passion - Ambition, passion for job
5. Other - Other reasons


In [ ]:
reason_categories = {
    'Want': "1",
    'Requirement': "2",
    'Exceptional': "3",
    'Passion': "4",
    'Other': "5"
}

def get_reason_category(x):
    for key, value in reason_categories.items():
        if x == value:
            return key

    return "Unknown"

df_col = "PUFC29_WWM48H"
df_new_col = "Work48H"

In [ ]:
df = one_hot_encode(df, "PUFC29_WWM48H", "Work48H", get_reason_category)

### PUFNEWEMPSTAT
One hot encode the categories if the individual is employed, not employed, or not in workforce.

In [ ]:
workforce_categories = {
    'Employed': "1",
    'Unemployed': "2",
    'NotInWorkForce': "3",
}

def get_workforce_category(x):
    for key, value in workforce_categories.items():
        if x == value:
            return key

    return "Unknown"


In [ ]:
df = one_hot_encode(df, "PUFNEWEMPSTAT", "Employment_Status", get_workforce_category)

### PUFC17_NATEM
One hot encode the categories if permanent, short-term, or different employer.

In [ ]:
nature_categories = {
    "Permanent": "1",
    "Short-Term": "2",
    "Different_Employer": "3",
}

def get_nature_category(x):
    for key, value in nature_categories.items():
        if x == value:
            return key

    return "Unknown"

In [ ]:
df = one_hot_encode(df, "PUFC17_NATEM", "NatureOfEmployment", get_nature_category)

In [ ]:
df.drop(columns=["PUFC07_GRADE", "Education", "PUFC16_PKB", "KindOfBusiness", "PUFC29_WWM48H", "Work48H", "PUFNEWEMPSTAT", "Employment_Status", "PUFC17_NATEM", "NatureOfEmployment"], inplace=True)

In [ ]:
df

### Label Encoding

Label encode remaining variables with more than 2 categories that can be ordered.

In [ ]:
df["Hours_Work_Total"] = pd.to_numeric(df["PUFC28_THOURS"], errors="coerce").astype('int64')

In [ ]:
encoder = LabelEncoder()
df["Marital_Status"] = encoder.fit_transform(df["PUFC06_MSTAT"]).astype('int64')
df["Working_Hours_Per_Week"] = encoder.fit_transform(df["PUFC18_PNWHRS"]).astype('int64')
df["Class_Of_Worker"] = encoder.fit_transform(df["PUFC23_PCLASS"]).astype('int64')
df["Payment_Basis"] = encoder.fit_transform(df["PUFC24_PBASIS"]).astype('int64')

In [ ]:
df.drop(columns=["PUFC06_MSTAT", "PUFC18_PNWHRS", "PUFC23_PCLASS", "PUFC24_PBASIS", "PUFC28_THOURS"], inplace=True)

Rename Remaining Variables

In [ ]:
df = df.rename(columns={
    "PUFHHSIZE": "Household_Size",
    "PUFC03_REL": "Relationship",
    "PUFC05_AGE": "Age",
})

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates()

In [ ]:
generate_report(df, "report-encoded")

## Data Transformation
This is to apply different techniques to normalize the data and make it more suitable for machine learning models.

In [ ]:
from data_transformation import DataTransformer

DT = DataTransformer(df)
DT.transform_data()

This shows the skewed columns that have been transformed.

In [ ]:
DT.scale_features()

In [ ]:
DT.visualize()

This shows the scaled columns after transformation.

In [ ]:
transformed_df = DT.scaled_df

In [ ]:
DT.visualize(scaled=True)

## Identify features (X) and labels (y)
Separate the labels from the features for training. This is to ensure the models are properly trained.

In [ ]:
target_variable = "Class_Of_Worker"
X = transformed_df.drop(columns=[target_variable]).to_numpy()
y = transformed_df[target_variable].to_numpy()
X.shape, y.shape

## Train-Test Split
This section is dedicated to splitting the dataset into training and testing sets. For consistency, the data will be split using a random seed of 42. and a test_size of 0.2.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Handling Class Imbalance
To handle class imbalance, we can use the Synthetic Minority Over-sampling Technique (SMOTE) to oversample the minority classes. This technique generates synthetic samples to balance the class distribution.

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Initial Random Forest Model

In [ ]:
target_column = "Class_Of_Worker"
X, y = preprocess_data(df, target_column)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
initial_forest_model = train_random_forest(X_train, y_train)

In [ ]:
initial_forest_model = train_random_forest(X_train_resampled, y_train_resampled)

In [ ]:
evaluate_model(initial_forest_model, X_test, y_test)

### Improving Random Forest Performance

#### Gradient Boosting and Hyperparamter Tuning

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

In [ ]:
random_search = RandomizedSearchCV(
    gb_model, param_distributions=param_grid,
    n_iter=5, cv=2, scoring='accuracy', n_jobs=-1, verbose=2
)
random_search.fit(X_train, y_train)

In [ ]:
best_forest_model = random_search.best_estimator_

In [ ]:
evaluate_model(best_forest_model, X_test, y_test)

### K-NN Algorithm

##### Using K-NN as a machine learning model for classification in  with the target variable of **Class_Of_Worker** to identify if K-NN is an accurate model to use with hyperparameters.

Without hyperparameter tuning

In [ ]:
y = y.astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# KNN model (default k=5)
initial_knn_model = KNeighborsClassifier()
initial_knn_model.fit(X_train_scaled, y_train)  # Train classifier

# Predictions
y_pred = initial_knn_model.predict(X_test_scaled)

# Evaluation
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))

With hyperparameter tuning

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Hyperparameter tuning using Optuna for a faster and smarter searching, and automatiacally finding the best parameters.

In [ ]:
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 5, 50, step=5)  # Optimize k
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights='distance', metric='euclidean')
    return cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()

In [ ]:
study = optuna.create_study(direction='maximize')

In [ ]:
study.optimize(objective, n_trials=10)

# Print best parameters
print("Best hyperparameters:", study.best_params)

Best hyperparameter detected was n_neighbors = 10

In [ ]:
best_knn_model = KNeighborsClassifier(n_neighbors=10)
best_knn_model.fit(X_train, y_train)

In [ ]:
y_pred = best_knn_model.predict(X_test)

In [ ]:
print("\n" + classification_report(y_test, y_pred) + "\n")
print(confusion_matrix(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot using seaborn
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=initial_knn_model.classes_, yticklabels=initial_knn_model.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Artificial Neural Networks
Let us make predictions using an artificial neural network (ANN). We will first do so using default or commonly used parameters. Then, we will perform performance enhancing techniques to improve the model's performance.

## Import Necessary Modules
We will need to import the NeuralNetwork class from neural_network.py as well as other relevant modules


In [ ]:
import torch
import pandas as pd
from neural_network import  train_neural_network, evaluate_model, tune_hyperparameters

## Initialize the model

In [ ]:
"""
Idenfifying the target column and the features, as well as splitting the datainto training
and test sets are no longer necessary as they had already been done earlier in the notebook.

"""

# Get input size and number of classes
input_size = X_train.shape[1]
num_classes = len(set(y_train))

# Define initial model parameters
list_hidden = [64, 32]
activation = 'relu'
learning_rate = 0.01
epochs = 100


## Train the model
Perform an initial training of the model using the default parameters

In [ ]:
initial_nn_model = train_neural_network(X_train, y_train, input_size, list_hidden, num_classes, activation, epochs, learning_rate)

## Evaluate Model on Test Set

In [ ]:
evaluate_model(initial_nn_model, X_test, y_test)

## Initial Results
The model with default hyperparameters produced an accuracy of `74.8960%`. However, based on the classification report, classes 4 and 5 were never predicted at all (recall=0.00), indicating that they are underrepresented in the model's predictions. For now, let us perform hyperparameter tuning.

## Hyperparameter Tuning
Perform hyperparameter tuning using Optuna

In [ ]:
best_params = tune_hyperparameters(X_train, y_train, input_size, num_classes, n_trials=40)
print("Best Hyperparameters:", best_params)

## Results

Performing 40 trials of hyperparameter tuning with Optuna, the following values hyperparameters were found to be the best:
- num-layers: 2
- hidden_list [24, 101]
- learning_rate: 0.0081603746771058
- activation: 'relu'
- epochs: 161

## Retrain the Model
Apply the best hyperparameters to the same model

In [ ]:
# Redefine the hyperparemeters
list_hidden = [24, 101]
activation = 'relu'
learning_rate = 0.0081603746771058
epochs = 161

tuned_nn_model = train_neural_network(X_train, y_train, input_size, list_hidden, num_classes, activation, epochs, learning_rate)

## Evaluate the Retrained Model

In [ ]:
evaluate_model(tuned_nn_model, X_test, y_test)

## Results
The retrained model produced a test accuracy of `82.7619%`, which is higher than the initial accuracy of `74.8960%`. Let us perform a paired t-test to see if the hyperparameter tuning resulted in a significant model improvement.


## Statistical Tests

Random Forest Classifier

In [ ]:
y_pred_initial = initial_forest_model.predict(X_test)
y_pred_optimal = best_forest_model.predict(X_test)

get_statistical_significance(y_test, y_pred_initial, y_pred_optimal)

K-Nearest-Neighbors Classifier

In [ ]:
y_pred_initial = initial_knn_model.predict(X_test)
y_pred_optimal = best_knn_model.predict(X_test)

get_statistical_significance(y_test, y_pred_initial, y_pred_optimal)

Artificial Neural Network

In [ ]:
# Get predictions from both models
y_pred_initial = initial_nn_model.predict(torch.tensor(X_test, dtype=torch.float32)).numpy()
y_pred_optimal = tuned_nn_model.predict(torch.tensor(X_test, dtype=torch.float32)).numpy()

get_statistical_significance(y_test, y_pred_initial, y_pred_optimal)